# Torch Core

This module contains all the basic functions we need in other modules of the fastai library (split with [`core`](/core.html#core) that contains the ones not requiring pytorch). Its documentation can easily be skipped at a first read, unless you want to know what a given fuction does.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.torch_core import * 

## Global constants

`AdamW = partial(optim.Adam, betas=(0.9,0.99))` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L43">[source]</a></div>

`bn_types = (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L41">[source]</a></div>

`default_device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L42">[source]</a></div>

## Functions that operate conversions

In [ ]:
show_doc(flatten_model, full_name='flatten')

<h4 id="flatten"><code>flatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L123" class="source_link">[source]</a></h4>

> <code>flatten</code>(`m`)

Flattens all the layers of `m`. 

In [ ]:
show_doc(model2half)

<h4 id="model2half"><code>model2half</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L178" class="source_link">[source]</a></h4>

> <code>model2half</code>(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Convert `model` to half precision except the batchnorm layers.  

In [ ]:
show_doc(np2model_tensor)

<h4 id="np2model_tensor"><code>np2model_tensor</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L220" class="source_link">[source]</a></h4>

> <code>np2model_tensor</code>(`a`)

Tranform numpy array `a` to a tensor of the same type.  

In [ ]:
show_doc(requires_grad, doc_string=False)

<h4 id="requires_grad"><code>requires_grad</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L99" class="source_link">[source]</a></h4>

> <code>requires_grad</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `b`:`Optional`\[`bool`\]=`None`) → `Optional`\[`bool`\]

If `b` is None, returns the [`requires_grad`](/torch_core.html#requires_grad) state of the first layer of `m`. Otherwise, sets `requires_grad=b` in all children of `m`.

In [ ]:
show_doc(tensor)

<h4 id="tensor"><code>tensor</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L63" class="source_link">[source]</a></h4>

> <code>tensor</code>(`x`:`Any`, `rest`) → `Tensor`

Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly  

Ensures `x` is a torch `Tensor`.

In [ ]:
show_doc(to_data)

<h4 id="to_data"><code>to_data</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L79" class="source_link">[source]</a></h4>

> <code>to_data</code>(`b`:`ItemsList`)

Recursively map lists of items in `b ` to their wrapped data.  

In [ ]:
show_doc(to_detach)

<h4 id="to_detach"><code>to_detach</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L74" class="source_link">[source]</a></h4>

> <code>to_detach</code>(`b`:`Tensors`)

Recursively detach lists of tensors in `b `  

In [ ]:
show_doc(to_device)

<h4 id="to_device"><code>to_device</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L89" class="source_link">[source]</a></h4>

> <code>to_device</code>(`b`:`Tensors`, `device`:[`device`](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device))

Ensure `b` is on `device`.  

In [ ]:
show_doc(to_half, doc_string=False)

<h4 id="to_half"><code>to_half</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L168" class="source_link">[source]</a></h4>

> <code>to_half</code>(`b`:`Collection`\[`Tensor`\]) → `Collection`\[`Tensor`\]

Put the input of the batch `b` in half precision.

In [ ]:
show_doc(to_np)

<h4 id="to_np"><code>to_np</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L235" class="source_link">[source]</a></h4>

> <code>to_np</code>(`x`)

Convert `x` to a numpy array.

## Functions to deal with model initialization

In [ ]:
show_doc(apply_init)

<h4 id="apply_init"><code>apply_init</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L194" class="source_link">[source]</a></h4>

> <code>apply_init</code>(`m`, `init_func`:`LayerFunc`)

Initialize all non-batchnorm layers of `m` with `init_func`.  

In [ ]:
show_doc(apply_leaf)

<h4 id="apply_leaf"><code>apply_leaf</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L188" class="source_link">[source]</a></h4>

> <code>apply_leaf</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `f`:`LayerFunc`)

Apply `f` to children of `m`.  

In [ ]:
show_doc(cond_init)

<h4 id="cond_init"><code>cond_init</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L182" class="source_link">[source]</a></h4>

> <code>cond_init</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `init_func`:`LayerFunc`)

Initialize the non-batchnorm layers of `m` with `init_func`  

In [ ]:
show_doc(in_channels)

<h4 id="in_channels"><code>in_channels</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L198" class="source_link">[source]</a></h4>

> <code>in_channels</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `List`\[`int`\]

Return the shape of the first weight layer in `m`.  

## Functions to get information of a model

In [ ]:
show_doc(children)

<h4 id="children"><code>children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L111" class="source_link">[source]</a></h4>

> <code>children</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `ModuleList`

Get children of `m`.  

In [ ]:
show_doc(first_layer)

<h4 id="first_layer"><code>first_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L124" class="source_link">[source]</a></h4>

> <code>first_layer</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Retrieve first layer in a module `m`.  

In [ ]:
show_doc(last_layer)

<h4 id="last_layer"><code>last_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L128" class="source_link">[source]</a></h4>

> <code>last_layer</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Retrieve last layer in a module `m`.  

In [ ]:
show_doc(num_children)

<h4 id="num_children"><code>num_children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L115" class="source_link">[source]</a></h4>

> <code>num_children</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `int`

Get number of children modules in module `m`.  

In [ ]:
show_doc(range_children)

<h4 id="range_children"><code>range_children</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L119" class="source_link">[source]</a></h4>

> <code>range_children</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `Iterator`\[`int`\]

Return iterator of len of children of `m`.  

In [ ]:
show_doc(trainable_params)

<h4 id="trainable_params"><code>trainable_params</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L106" class="source_link">[source]</a></h4>

> <code>trainable_params</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `ParamList`

Return list of trainable params in `m`.  

## Functions to deal with BatchNorm layers

In [ ]:
show_doc(bn2float)

<h4 id="bn2float"><code>bn2float</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L172" class="source_link">[source]</a></h4>

> <code>bn2float</code>(`module`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

If `module` is batchnorm don't use half precision.  

In [ ]:
show_doc(set_bn_eval)

<h4 id="set_bn_eval"><code>set_bn_eval</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L161" class="source_link">[source]</a></h4>

> <code>set_bn_eval</code>(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))

Set bn layers in eval mode for all recursive children of `m`.  

In [ ]:
show_doc(split_bn_bias)

<h4 id="split_bn_bias"><code>split_bn_bias</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L150" class="source_link">[source]</a></h4>

> <code>split_bn_bias</code>(`layer_groups`:`ModuleList`) → `ModuleList`

Sort each layer in  `layer_groups` into batchnorm (`bn_types`) and non-batchnorm groups.  

## Other functions

In [ ]:
show_doc(calc_loss)

<h4 id="calc_loss"><code>calc_loss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L204" class="source_link">[source]</a></h4>

> <code>calc_loss</code>(`y_pred`:`Tensor`, `y_true`:`Tensor`, `loss_func`:`LossFunction`)

Calculate loss between `y_pred` and `y_true` using `loss_class` and `bs`.  

In [ ]:
show_doc(data_collate)

<h4 id="data_collate"><code>data_collate</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L95" class="source_link">[source]</a></h4>

> <code>data_collate</code>(`batch`:`ItemsList`) → `Tensor`

Convert `batch` items to tensor data.  

In [ ]:
show_doc(model_type)

<h4 id="model_type"><code>model_type</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L214" class="source_link">[source]</a></h4>

> <code>model_type</code>(`dtype`)

Return the torch type corresponding to `dtype`.  

In [ ]:
show_doc(np_address)

<h4 id="np_address"><code>np_address</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L70" class="source_link">[source]</a></h4>

> <code>np_address</code>(`x`:`ndarray`) → `int`

Address of `x` in memory  

In [ ]:
show_doc(split_model, doc_string=False)

<h4 id="split_model"><code>split_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L139" class="source_link">[source]</a></h4>

> <code>split_model</code>(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `splits`:`Collection`\[`Union`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `ModuleList`\]\], `want_idxs`:`bool`=`False`)

Splits the `model` according to the layer in `splits`. If `splits` are layers, the model is split at those (not included) sequentially. If `want_idxs` is True, the corresponding indexes are returned. If `splits` are lists of layers, the model is split according to those.

In [ ]:
show_doc(split_model_idx)

<h4 id="split_model_idx"><code>split_model_idx</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L132" class="source_link">[source]</a></h4>

> <code>split_model_idx</code>(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `idxs`:`Collection`\[`int`\]) → `ModuleList`

Split `model` according to the indices in `idxs`.  

In [ ]:
show_doc(trange_of)

<h4 id="trange_of"><code>trange_of</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L233" class="source_link">[source]</a></h4>

> <code>trange_of</code>(`x`)

Return a tensor from a range that has the same length as `x`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(tensor__array__)

<h4 id="tensor__array__"><code>tensor__array__</code><a href="https://github.com/fastai/fastai/blob/master/fastai/torch_core.py#L238" class="source_link">[source]</a></h4>

> <code>tensor__array__</code>(`dtype`=`None`)

## New Methods - Please document or move to the undocumented section